<a href="https://colab.research.google.com/github/martinpius/Computer-Vission/blob/main/Transfer_learning_in_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import tensorflow as tf
  print(f"You are on CoLaB with tensorflow version: {tf.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n>>>please correct {type(e)}")
  COLAB = False
def time_fmt(t:float = 123.871)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h}: {m:>02}: {s:>05.2f}"
print(f">>>time testing, please wait...\n>>>time elapse: {time_fmt()}")


Mounted at /content/drive
You are on CoLaB with tensorflow version: 2.4.1
>>>time testing, please wait...
>>>time elapse: 0: 02: 03.00


In [86]:
#In this notebook we discuss transfer learning technique with application of cifar10 dataset:


In [87]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import time, sys
from IPython import embed
from skimage.transform import resize


In [88]:
#Reshaping the images into required resnet-architecture (244,244,3)
def img_reshape(image):
  image = image.astype('float32')
  out_img = tf.keras.applications.resnet50.preprocess_input(image)
  return out_img


In [89]:
#Loading the data from keras:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [90]:
print(f"x_train_shape: {x_train.shape}, y_train_shape: {y_train.shape}\nx_test_shape: {x_test.shape}, y_test_shape: {y_test.shape}")



x_train_shape: (50000, 32, 32, 3), y_train_shape: (50000, 1)
x_test_shape: (10000, 32, 32, 3), y_test_shape: (10000, 1)


In [91]:
#Applying preprocessing function 
x_train, x_test = img_reshape(x_train), img_reshape(x_test)

In [92]:
y_train, y_test = tf.keras.utils.to_categorical(y_train, 10), tf.keras.utils.to_categorical(y_test, 10)

In [93]:
print(f"x_train_shape: {x_train.shape}, y_train_shape: {y_train.shape}\nx_test_shape: {x_test.shape}, y_test_shape: {y_test.shape}")

x_train_shape: (50000, 32, 32, 3), y_train_shape: (50000, 10)
x_test_shape: (10000, 32, 32, 3), y_test_shape: (10000, 10)


In [94]:
#Get the pre-trained weights from renet50 (inputs image is 224 by 224)
def _extractor_(inputs):
  ft_ext = tf.keras.applications.resnet.ResNet50(include_top = False, weights = 'imagenet')(inputs)
  return ft_ext

In [95]:
#Modify the model (define classifier to suit your need: here with 10 classes)
def classfier(inputs):
  '''we modify resnet50 by excluding and add few layers with the last=classifier'''
  x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(units = 1024, activation = 'relu')(x)
  x = tf.keras.layers.Dense(units = 512, activation = 'relu')(x)
  x = tf.keras.layers.Dense(units = 10, activation = 'softmax')(x)
  return x

#We need to reshape the images from (32,32,3) to (224,224,3). This can be achieved 
#through various procidures but the easy shortcut is to upsampling(Deconvolution):
def trf_model(inputs):
  x = tf.keras.layers.UpSampling2D(size = (7,7))(inputs)
  resnet_ftx = _extractor_(x)
  trf_out = classfier(resnet_ftx)
  return trf_out

#We now define our model and compile it ready for training:
def final_model():
  inputs = tf.keras.layers.Input(shape = (32,32,3))
  outputs = trf_model(inputs)
  model = tf.keras.models.Model(inputs = inputs, outputs = outputs)
  model.compile(loss = 'categorical_crossentropy',
                optimizer = 'Adam', metrics = ['accuracy'])
  return model

#Display the summary of our model
model = final_model()
display(model.summary())

#training our model:
tic = time.time()
model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 5, batch_size = 64, verbose = 2)
toc = time.time()
print(f"\n>>>time elapse: {time_fmt(toc - tic)}")

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, None, None, 2048)  23587712  
_________________________________________________________________
global_average_pooling2d_7 ( (None, 2048)              0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_24 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dense_25 (Dense)             (None, 512)               5248

None

Epoch 1/5
782/782 - 267s - loss: 0.7623 - accuracy: 0.7425 - val_loss: 0.8127 - val_accuracy: 0.7351
Epoch 2/5
782/782 - 259s - loss: 0.4277 - accuracy: 0.8547 - val_loss: 1.4179 - val_accuracy: 0.6467
Epoch 3/5
782/782 - 259s - loss: 0.3086 - accuracy: 0.8951 - val_loss: 0.4835 - val_accuracy: 0.8476
Epoch 4/5
782/782 - 259s - loss: 0.2317 - accuracy: 0.9207 - val_loss: 0.4101 - val_accuracy: 0.8612
Epoch 5/5
782/782 - 259s - loss: 0.1815 - accuracy: 0.9379 - val_loss: 0.5413 - val_accuracy: 0.8320

>>>time elapse: 0: 21: 44.00
